<a href="https://colab.research.google.com/github/nicolenair/capstone/blob/master/Capstone_LL_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

In [8]:
# !pip install import_ipynb
# !pip install nltk

# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
import requests

from bs4 import BeautifulSoup
import requests
import nltk
import numpy as np
import pandas as pd
import os
import re
import os
import import_ipynb
import anno

importing Jupyter notebook from anno.ipynb
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
books = ["gables", "avonlea", "island", "dreams"]
urls = ["https://www.gutenberg.org/files/45/45-0.txt", "https://www.gutenberg.org/files/47/47-0.txt", "https://www.gutenberg.org/files/51/51-0.txt", "http://www.gutenberg.org/cache/epub/544/pg544.txt"]

for i in range(len(books)):
    page = requests.get(urls[i])
    file = open("{}.txt".format(books[i]) , "w+")
    file.write(page.text)
    file.close()

file = open("gables.txt", "r")
gables = file.read()
file = open("avonlea.txt", "r")
avonlea = file.read()
file = open("island.txt", "r")
island = file.read()
file = open("dreams.txt", "r")
dreams = file.read()

In [0]:
full_para_list_gables = anno.paragraph_list(gables)
full_para_list_dreams = anno.paragraph_list(dreams)

In [13]:
full_para_list_dreams[9][67]

'"Don\'t let us judge her till we know," pleaded Anne.  "I don\'t believe her case is so ordinary.  You will understand her fascination when you meet her, Gilbert.  It is a thing quite apart from her beauty.  I feel that she possesses a rich nature, into which a friend might enter as into a kingdom; but for some reason she bars every one out and shuts all her possibilities up in herself, so that they cannot develop and blossom.  There, I\'ve been struggling to define her to myself ever since I left her, and that is the nearest I can get to it.  I\'m going to ask Miss Cornelia about her."'

In [0]:
import nltk

word_list_dreams = dreams.split()
# word_list_dreams[:10]
postagged_word_list_dreams = nltk.pos_tag(word_list_dreams)

list_of_tags = [i[1] for i in postagged_word_list_dreams]
x_tagname = []
y_tagcount = []
for i in list(set(list_of_tags)):
  y_tagcount.append(list_of_tags.count(i))
  x_tagname.append(i)
  
full_dreams_tags = pd.DataFrame({"tagname":x_tagname, "tagcount":y_tagcount})

In [17]:
words_anne_dreams = []
target_dreams = anno.return_target("dreams")
target_gables = anno.return_target("gables")
for i in range(len(target_dreams)):
  for e in range(len(target_dreams[i])):
#      print(i, e)
     if target_dreams[i][e]==1:
        words_anne_dreams.extend(full_para_list_dreams[i][e].split())
        
tag_anne_dreams = nltk.pos_tag(words_anne_dreams)

list_of_tags_anne_dreams = [i[1] for i in tag_anne_dreams]
x_tagname = []
y_tagcount = []
for i in list(set(list_of_tags_anne_dreams)):
  y_tagcount.append(list_of_tags_anne_dreams.count(i))
  x_tagname.append(i)
  
# full_gables_tags.loc["anne_tagname"] = x_tagname
# full_gables_tags.loc["anne_tagcount"] = y_tagcount
full_dreams_tags["anne_tagcount"] = 0
for i in range(len(full_dreams_tags)):
  if full_dreams_tags["tagname"][i] in x_tagname:
    full_dreams_tags["anne_tagcount"][i] = y_tagcount[x_tagname.index(full_dreams_tags["tagname"][i])]
    
    
full_dreams_tags["anne_tagcount"] = full_dreams_tags["anne_tagcount"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [0]:
import nltk

word_list_gables = gables.split()
# word_list_dreams[:10]
postagged_word_list_gables = nltk.pos_tag(word_list_gables)

list_of_tags = [i[1] for i in postagged_word_list_gables]
x_tagname = []
y_tagcount = []
for i in list(set(list_of_tags)):
  y_tagcount.append(list_of_tags.count(i))
  x_tagname.append(i)
  
full_gables_tags = pd.DataFrame({"tagname":x_tagname, "tagcount":y_tagcount})

In [19]:
words_anne_gables = []
for i in range(len(target_gables)):
  for e in range(len(target_gables[i])):
     # print(i, e)
     if target_gables[i][e]==1:
        words_anne_gables.extend(full_para_list_gables[i][e].split())
        
tag_anne_gables = nltk.pos_tag(words_anne_gables)

list_of_tags_anne = [i[1] for i in tag_anne_gables]
x_tagname = []
y_tagcount = []
for i in list(set(list_of_tags_anne)):
  y_tagcount.append(list_of_tags_anne.count(i))
  x_tagname.append(i)
  
# full_gables_tags.loc["anne_tagname"] = x_tagname
# full_gables_tags.loc["anne_tagcount"] = y_tagcount
full_gables_tags["anne_tagcount"] = 0
for i in range(len(full_gables_tags)):
  if full_gables_tags["tagname"][i] in x_tagname:
    full_gables_tags["anne_tagcount"][i] = y_tagcount[x_tagname.index(full_gables_tags["tagname"][i])]
    
    
full_gables_tags["anne_tagcount"] = full_gables_tags["anne_tagcount"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
for i in full_gables_tags["tagname"]:
  if i not in full_dreams_tags["tagname"]:
    full_dreams_tags["tagname"][i] = 0.01

#   if full_gables_tags["tagcount"]
    
for i in full_dreams_tags["tagname"]:
  if i not in full_gables_tags["tagname"]:
    full_gables_tags["tagname"][i] = 0.01
    
full_dreams_tags.loc[full_dreams_tags["tagcount"]==0, "tagcount"] = 0.0001
full_gables_tags.loc[full_gables_tags["tagcount"]==0, "tagcount"] = 0.0001
full_dreams_tags.loc[full_dreams_tags["anne_tagcount"]==0, "anne_tagcount"] = 0.0001
full_gables_tags.loc[full_gables_tags["anne_tagcount"]==0, "anne_tagcount"] = 0.0001


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [0]:
# full_dreams_tags

In [21]:
from scipy.stats import chi2_contingency
print(chi2_contingency(full_gables_tags.iloc[:,1:])[:2])
print(chi2_contingency(full_dreams_tags.iloc[:,1:])[:2])
print(chi2_contingency(pd.DataFrame({"0":np.array(full_gables_tags.iloc[:,2]), "1": np.array(full_dreams_tags.iloc[:,2])}))[:2])
print(chi2_contingency(pd.DataFrame({"0":np.array(full_gables_tags.iloc[:,1]), "1": np.array(full_dreams_tags.iloc[:,1])}))[:2])
print(chi2_contingency(pd.DataFrame({"0":np.array(full_gables_tags.iloc[:,1]), "1": np.array(full_dreams_tags.iloc[:,2])}))[:2])
print(chi2_contingency(pd.DataFrame({"0":np.array(full_gables_tags.iloc[:,2]), "1": np.array(full_dreams_tags.iloc[:,1])}))[:2])

(92.6337504561383, 6.953700226937056e-07)
(120.58595134123262, 4.663514636757808e-11)
(380.2604674150601, 4.57852218142955e-59)
(6321.129300735027, 0.0)
(3444.8674530366575, 0.0)
(2235.890803184948, 0.0)


In [22]:
full_dreams_tags

,tagname,tagcount,anne_tagcount
0,$,2.0,0.0001
1,RBR,55.0,3.0000
2,CD,529.0,27.0000
3,POS,51.0,6.0000
4,NNS,1933.0,118.0000
5,'',19.0,1.0000
6,NNPS,24.0,0.0001
7,NNP,8411.0,545.0000
8,CC,3093.0,134.0000
9,RB,3789.0,237.0000


In [0]:
pd.DataFrame({"0":np.array(full_gables_tags.iloc[:,2]), "1": np.array(full_dreams_tags.iloc[:,2])})

,0,1
0,1,0
1,1,0
2,0,237
3,261,469
4,491,123
5,105,118
6,99,83
7,65,388
8,7,6
9,414,1


0       0
1       0
2     237
3     469
4     123
5     118
6      83
7     388
8       6
9       1
10    674
11    134
12     34
13    545
14      3
15     89
16     12
17     23
18     97
19    262
20     28
21     69
22      3
23    112
24    311
25      6
26     16
27    254
28      7
29      0
30    197
31      6
32      0
33      0
34      5
35    394
36     27
Name: anne_tagcount, dtype: int64